In [1]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from hep_VQAE import data_preprocessing as dp
import tensorflow as tf
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
import time

import warnings
warnings.filterwarnings('ignore')

2022-08-01 12:33:40.046026: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-01 12:33:40.046047: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Circuit

In [17]:
DATA_QBITS = 4
LATENT_QBITS = 2
TRASH_QBITS = DATA_QBITS - LATENT_QBITS
TOTAL_QBITS = DATA_QBITS + TRASH_QBITS + 1

In [18]:
dev1 = qml.device('lightning.qubit', wires=TOTAL_QBITS)

In [31]:
@qml.qnode(dev1, diff_method="parameter-shift")
def circuit(enc_params, data):
    qml.AngleEmbedding(data, wires=range(DATA_QBITS))
    qml.BasicEntanglerLayers(enc_params, wires=range(DATA_QBITS))
    qml.Hadamard(wires=TOTAL_QBITS-1)
    for i in range(TRASH_QBITS):
        #c += cirq.ControlledGate(sub_gate=cirq.SWAP, num_controls=1).on(swap_qbit, reference_qbits[i], network_qbits[num_data_qbits - num_latent_qbits:][i])
        qml.CSWAP(wires=[TOTAL_QBITS - 1, LATENT_QBITS + i, DATA_QBITS + i])
    qml.Hadamard(wires=TOTAL_QBITS-1)
    return qml.expval(qml.PauliZ(TOTAL_QBITS-1))

In [32]:
#fig, ax = qml.draw_mpl(circuit)([[1,1,1,1]],[0.1,0.24,0.3,0.14])
#fig.show()

In [33]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

In [34]:
def filter_36(x, y):
    keep = (y == 3) | (y == 6)
    x, y = x[keep], y[keep]
    y = y == 3
    return x,y

x_train, y_train = filter_36(x_train, y_train)
x_test, y_test = filter_36(x_test, y_test)

In [35]:
DOWNSCALE = 2
DATA_DIMENSION = DOWNSCALE*DOWNSCALE
LATENT_BITS = 2

x_train, x_test = dp.PCA_reduce(x_train, DATA_DIMENSION, val_data=x_test)

In [36]:
x_train_threes = np.array(x_train[y_train==True])[:1500]
x_train = np.array(x_train[y_train==False])[:1500]

x_test_threes = np.array(x_test[y_test==True])
x_test = np.array(x_test[y_test==False])

In [37]:
LAYERS = 5

def cost(params):
    loss = 0.0
    for i in range(len(x_train)):
        f = circuit(params, x_train[i])
        loss = loss + (1 - f) ** 2
    return loss / len(x_train)

def cost_sample(params, single_sample):
    return (1 - circuit(params, single_sample)) ** 2

In [38]:
def iterate_minibatches(data, batch_size):
    for start_idx in range(0, data.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield data[idxs]

In [85]:
learning_rate = 0.2
epochs = 2
batch_size = 64 

In [86]:
#opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)
opt = qml.QNGOptimizer(learning_rate)
# opt = GradientDescentOptimizer(learning_rate)

In [87]:
weight_shapes = {"weights": (LAYERS, DATA_QBITS)}
params = np.random.uniform(size=weight_shapes["weights"], requires_grad=True)

In [88]:
start = time.time()

for it in range(epochs):
    # for j,Xbatch in enumerate(iterate_minibatches(x_train, batch_size=batch_size)):
    start_b = time.time()
    for j, sample in enumerate(x_train):        
        sample = np.array(sample, requires_grad=False)
        cost_fn = lambda p: cost_sample(p, sample)
        metric_fn = lambda p: qml.metric_tensor(circuit, approx="block-diag")(p, sample)
        params = opt.step(cost_fn, params, metric_tensor_fn=metric_fn)
        print(j, end="\r")
        if j % 100 == 0:
            end_b = time.time()
            loss = cost(params)
            print(f"Step: {j} | Loss: {loss:.3} | avg step time {(end_b - start_b) / 100 :.3}")
            start_b = time.time()

    loss = cost(params)
    
    print(f"Epoch: {it} | Loss: {loss} |")
    
end = time.time()

print(f"Time for {epochs} epochs with {LAYERS} layers: {end - start}")

Step: 0 | Loss: 0.0586 | avg step time 0.000922
Step: 100 | Loss: 0.000621 | avg step time 0.176
Step: 200 | Loss: 0.000551 | avg step time 0.182
Step: 300 | Loss: 0.00051 | avg step time 0.181
Step: 400 | Loss: 0.000543 | avg step time 0.18
Step: 500 | Loss: 0.000546 | avg step time 0.182
Step: 600 | Loss: 0.000513 | avg step time 0.181
Step: 700 | Loss: 0.000584 | avg step time 0.182
Step: 800 | Loss: 0.000574 | avg step time 0.186
Step: 900 | Loss: 0.000592 | avg step time 0.197
Step: 1000 | Loss: 0.000565 | avg step time 0.204
Step: 1100 | Loss: 0.000649 | avg step time 0.201
Step: 1200 | Loss: 0.00069 | avg step time 0.202
Step: 1300 | Loss: 0.000538 | avg step time 0.2
Step: 1400 | Loss: 0.000793 | avg step time 0.203
Epoch: 0 | Loss: 0.0005629334175304904 |
Step: 0 | Loss: 0.000557 | avg step time 0.00303
Step: 100 | Loss: 0.000574 | avg step time 0.199
Step: 200 | Loss: 0.000549 | avg step time 0.203
Step: 300 | Loss: 0.00051 | avg step time 0.203
Step: 400 | Loss: 0.000543 | a